In [ ]:
# Import packages
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import math
import datetime as dt
import yfinance as yf
import random

# Obtaining Data

In [ ]:
# Function to extract stock data we want
def get_stock_data(tickers, num_days_back, info_type):
    """Takes in a list of stock tickers, the number of days into
    the past we want data from and the type of price information, and 
    returns a Pandas dataframe with the stock data"""
    
    end_date = dt.datetime.now()
    start_date = end_date - dt.timedelta(days = num_days_back)
    
    # Initialize a dataframe to store stock data
    df = pd.DataFrame()
    
    # For each stock, download the desired information
    for ticker in tickers:
        data = yf.download(ticker, start = start_date, end = end_date)
        df[ticker] = data[info_type]
    
    return df

In [ ]:
# Sustainable Stocks/Tickers we want to analyze
stocks_sustainable = ["AQN", "BEPC", "CEG", "CSIQ", "DQ", "GE", "IBDRY", "JKS", "NEE", "VWDRY"]

# Unsustainable companies tickers
stocks_unsustainable = ["EQNR", "BP", "0857.HK", "COP", "FP.VI", "SHEL", "CVX", "SR", "ENEL.MI"]

df_sustain = get_stock_data(tickers = stocks_sustainable, num_days_back = 1000, info_type = "Adj Close")
df_unsustain = get_stock_data(tickers = stocks_unsustainable, num_days_back = 1000, info_type = "Adj Close")

# Trading Strategies

## Baseline strategy: Random Trading

In [ ]:
def simulate_trade(ticker_symbol, initial_balance, num_days, data = None, description = True):
    """Takes in ticker of a chosen stock, initial balance to trade with,
    and number of days to trade for, performs the random buying and selling strategy
    for for the most recent days possible, and returns the description of trades and final
    account balance"""
    
    # Download historical stock data if not given
    if data is None:
        stock_data = yf.download(ticker_symbol, 
                        start=dt.datetime.now() - dt.timedelta(days=num_days),
                        end=dt.datetime.now())
    elif isinstance(data, pd.core.frame.DataFrame):
        stock_data = data
    else:
        raise ValueError("Invalid type for 'data'. Expected DataFrame or None.")
    
    # Initialize variables for account balance and stock quantity held
    balance = initial_balance
    stock_quantity = 0

    # Define the trades for each day
    for day in range(1, len(stock_data) + 1):
        action = random.choice(['buy', 'sell'])

        # Get the stock price for the current day
        current_price = stock_data['Adj Close'][day - 1]

        if action == 'buy':
            # Randomly determine the quantity to buy
            buy_quantity = random.randint(1, 10)

            total_cost = buy_quantity * current_price

            if total_cost <= balance:
                # Perform the purchase
                stock_quantity += buy_quantity
                balance -= total_cost
                
                # Describe trade if specified
                if description == True:
                    print(f"Day {day}: Bought {buy_quantity} stocks at ${current_price:.2f} each.")

        elif action == 'sell' and stock_quantity > 0:
            # Randomly determine the quantity to sell
            sell_quantity = random.randint(1, stock_quantity)

            total_earning = sell_quantity * current_price

            # Perform the sale
            stock_quantity -= sell_quantity
            balance += total_earning
            
            # Describe the trade if specified
            if description == True:
                print(f"Day {day}: Sold {sell_quantity} stocks at ${current_price:.2f} each.")

    # Sell remaining stocks on the last day
    if stock_quantity > 0:
        total_earning = stock_quantity * current_price
        balance += total_earning
        
        if description == True:
            print(f"Final day: Sold remaining {stock_quantity} stocks at ${current_price:.2f} each.")

    if description == True:
        print(f"Final balance: ${balance:.2f}")
    
    else:
        return balance

In [ ]:
# Test the function
simulate_trade("Shel", 10000, 20)

In [ ]:
# Get 30-day EQNR Data
df5 = stock_data = yf.download("EQNR", start = dt.datetime.now() - dt.timedelta(days = 30), end = dt.datetime.now())

In [ ]:
# Simulate many trading cycles (30-days)
trade_simulations_EQNR = [simulate_trade("EQNR", 100, 30, data = df5, description = False) for i in range(10000)]

In [ ]:
# Plot the trade simulations distribution
plt.figure(figsize=(10,4))
sns.displot(trade_simulations_EQNR)
plt.axvline(np.mean(trade_simulations_EQNR), color='b', linestyle='-')
plt.axvline(100, color='g', linestyle='--')
plt.title('Avg: $%s\nSD: $%s'%(round(np.mean(trade_simulations_EQNR),2), round(np.std(trade_simulations_EQNR),2)), fontsize=20)

## Models

### ARIMA

### GARCH

In [ ]:
from arch import arch_model
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
# Take just EQNR as an experiment
df1 = df_unsustain["EQNR"]

In [ ]:
returns = 100 * df1.pct_change().dropna()

print(returns)

In [ ]:
plt.figure(figsize = (10, 4))
plt.plot(returns)
plt.ylabel("Pct Return", fontsize = 16)
plt.title("EQNR Returns", fontsize = 20)

### PACF

In [ ]:
plot_pacf(returns**2)
plt.show() # Not very autocorrelated returns variance

In [ ]:
model_1 = arch_model(returns, p = 2, q = 2)

In [ ]:
model_fit = model_1.fit()

In [ ]:
# Insignificant -> not a useful choice
print(model_fit.summary())

## 1. AQN

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

In [ ]:
# Choose a stock from the dataset
df2 = df_sustain["AQN"]
df2.head(3)

In [ ]:
# Plot the prices -> non-stationary
plt.figure(figsize = (10, 4))
plt.plot(df2)
plt.ylabel("Price", fontsize = 16)
plt.title("AQN price", fontsize = 20)

In [ ]:
# First-Differences (returns)
df2_returns = 100*df2.pct_change().dropna()

# seems more stationary now
plt.figure(figsize = (10, 4))
plt.plot(df2_diff)
plt.ylabel("Returns", fontsize = 16)
plt.title("AQN Returns", fontsize = 20)

In [ ]:
# ACF to examine the MA part -> MA(1) seems likely
plot_acf(df2_diff)
plt.show

In [ ]:
# PACF to examine the AR part -> AR(1) seems likely
plot_pacf(df2_diff, method = "ywm")
plt.show

### Try ARMA(1,1)

In [ ]:
# Split data into training and testing datasets
train_end = dt.datetime(2023,9,1)
test_end = dt.datetime(2023,12,31)

train_data = df2_diff[:train_end]
test_data = df2_diff[train_end + dt.timedelta(days = 1):test_end]

In [ ]:
# Model define
model_1 = ARIMA(train_data, order = (1,0,1))

In [ ]:
# fit the model
model_1_fit = model_1.fit()

In [ ]:
# summary table -> only sigma2 significant, Does not make sense to use MA nor AR
print(model_1_fit.summary())

## 2. Shell

In [ ]:
# Load the dataset
df3 = df_unsustain["SHEL"]

# Plot the price
plt.figure(figsize = (10, 4))
plt.plot(df3)
plt.ylabel("Price", fontsize = 16)
plt.title("Shell Price", fontsize = 20)

In [ ]:
# calculate daily returns
returns_shell = 100 * df3.pct_change().dropna()

# Plot the returns
plt.figure(figsize = (10, 4))
plt.plot(returns_shell)
plt.ylabel("Returns", fontsize = 16)
plt.title("Shell Returns", fontsize = 20)

In [ ]:
# ACF
plot_acf(returns_shell)
plt.show

In [ ]:
# PACF -> not a good candidate for ARIMA
plot_pacf(returns_shell, method = "ywm")
plt.show

### Try GARCH

In [ ]:
# ACF
plot_acf(returns_shell**2)
plt.show

In [ ]:
# PACF
plot_pacf(returns_shell**2, method = "ywm")
plt.show

### GARCH(3,3)

In [ ]:
# Training and testing sets
# Split data into training and testing datasets
train_end_shell = dt.datetime(2023,9,1)
test_end_shell = train_end_shell + dt.timedelta(days=90)

train_data_shell = returns_shell[:train_end_shell]
test_data_shell = returns_shell[train_end_shell + dt.timedelta(days = 1):test_end_shell]

In [ ]:
# Define model
model_garch_shell = arch_model(train_data_shell, p=3, q=3)

In [ ]:
# Fit the model
fit_garch_shell = model_garch_shell.fit()

In [ ]:
print(fit_garch_shell.summary())

### Try GARCH(0,3)

In [ ]:
# Fit the model
model_garch_shell_2 = arch_model(train_data_shell, p=1, q=3)

garch_shell_2_fit = model_garch_shell_2.fit()

In [ ]:
print(garch_shell_2_fit.summary()) # seems like a better fit

### Predictions

In [ ]:
# Calculate the variance of Shell returns
#var_shell = np.var(returns_shell.dropna())
#print(var_shell)

In [ ]:
# Predict 7-days into the future
forecast_length = 90
prediction_shell = garch_shell_2_fit.forecast(horizon = forecast_length)

In [ ]:
# Plot the forecasted values
plt.figure(figsize=(10,4))
preds, = plt.plot(np.sqrt(prediction_shell.variance.values[-1, :]))
plt.title('Volatility Prediction', fontsize=20)
plt.legend(["90-day predicted Volatility"], fontsize=10)

In [ ]:
# Plot 7-day shell returns
plt.figure(figsize = (10, 4))
plt.plot(returns_shell[train_end_shell:(train_end_shell + dt.timedelta(days = 90))])
plt.ylabel("Price", fontsize = 16)
plt.title("Shell Price", fontsize = 20)